# Massive Post on Image Processing And Preparation For Deep Learning in Python
## Manipulate and transform images at will
![](https://cdn-images-1.medium.com/max/1080/1*mooOrVIu1RV-2UYjK_tJvw.jpeg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@pkprasad1996?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Prasad Panchakshari</a>
        on 
        <a href='https://unsplash.com/s/photos/kitten?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

## Introduction

We are here on a sad business. Very sad, indeed. We are here to learn how to take beautiful, breathtaking images and turn them into bunch of ugly little numbers so that they are more presentable to all those soulless, mindless machines. 

We will take animals and strip them of their color, making them black and white. Grab flowers with vivid colors and rob them of their beauty. We will look at disturbing images of XRays and see ways to make them even more disturbing. Sometimes, we might even have fun by drawing coins using a computer algorithm. 

In other words, we will learn how to perform image processing. And our library of honor will be Scikit-Image (Skimage) throughout the article.

For some reason, the font size in code cells are becoming smaller and smaller towards the end. You can read the notebook as a Medium article [here](https://towardsdatascience.com/massive-tutorial-on-image-processing-and-preparation-for-deep-learning-in-python-1-e534ee42f122?source=your_stories_page----------------------------------------).

## Setup

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage  # pip install scikit-image

warnings.filterwarnings("ignore")

In [ ]:
def show(image: np.ndarray, title="Image", cmap_type="gray", axis=False):
    """
    A function to display np.ndarrays as images
    """
    plt.imshow(image, cmap=cmap_type)
    plt.title(title)
    if not axis:
        plt.axis("off")
    plt.margins(0, 0)
    plt.show();

In [ ]:
def compare(
    original,
    filtered,
    title_filtered="Filtered",
    cmap_type="gray",
    axis=False,
    title_original="Original",
):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 8), sharex=True, sharey=True)

    ax1.imshow(original, cmap=cmap_type)
    ax1.set_title(title_original)

    ax2.imshow(filtered, cmap=cmap_type)
    ax2.set_title(title_filtered)

    if not axis:
        ax1.axis("off")
        ax2.axis("off")
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0.01)
    plt.margins(0, 0)
    plt.show();

## Table of Contents <small id="toc"></small>

#### 1. [Basics](#basics)
1. [What is an image?](#image)
2. [Image basics with NumPy and Skimage](#numpy_basics)
3. [Common transformations](#common_trans)
4. [Histogram of color channels](#histogram)

#### 2. [Filters](#filters)
1. [Manual thresholding](#threshold_manual)
2. [Thresholding - global](#global)
3. [Thresholding - local](#local)
4. [Edge detection](#edge)
5. [Smoothing](#smooth)
6. [Contrast enhancement](#contrast)
7. [Transformations](#transformations)

#### 3. [Image restoration and enhancement](#restoration)
1. [Inpainting](#inpainting)
2. [Noise📣](#noise)
3. [Reducing noise - denoising](#denoise)
4. [Superpixels and Segmentation](#superpixel)
5. [Contours](#contour)

#### 4. [Advanced operations](#advanced)
1. [Edge detection](#edge2)
2. [Corner detection](#corner)

## Basics <small id="basics"></small>

### 1. What is an image? <small id="image"></small>

Image data is probably the most common after text. So, how does a computer understand that selfie of you in front the Eiffel Tower?

It uses a grid of small square units called pixels. A single pixel covers a small area and has a value that represents color. The more pixels in an image, the higher its quality and more memory it takes to store. 

That's it. Image processing is mostly about manipulating these individual pixels (or sometimes, groups of them) so that computer vision algorithms can extract more information from them. 

<a href="#toc">Back to top🔝</a>

### 2. Image basics with NumPy and Skimage <small id="numpy_basics"></small>

Images are loaded as NumPy ndarrays in both Matplotlib and Skimage. 

In [ ]:
from skimage.io import imread  # pip install scikit-image

image = imread("../input/notebook-images/colorful_scenery.jpg")

type(image)

As always, NumPy arrays bring flexibility, speed and power into the game. Image processing is no different. 

Ndarrays make it easy to retrieve general details about the image, like its dimensions:

In [ ]:
image.shape

In [ ]:
image.ndim

In [ ]:
# The number of pixels
image.size  # 853 * 1280 * 3

Our hidden `image` is 853 pixels in height and 1280 in width. The third dimension denotes the value of the RGB (red, green, blue) color channel. Most common images formats are in 3D. 

You can retrieve individual pixel values via regular NumPy indexing. Below, we try to index the image to retrieve each of the three color channels:

In [ ]:
red = image[:, :, 0]

compare(image, red, "Red Channel of the Image", cmap_type="Reds_r")

In [ ]:
green = image[:, :, 1]

compare(image, green, "Green Channel of the Image", "Greens_r")

In [ ]:
blue = image[:, :, 2]

compare(image, blue, "Blue Channel of the Image", "Blues_r")

0 for red, 1 for green and 2 for blue channel - easy enough. 

I've created two functions, [`show`](https://gist.github.com/BexTuychiev/e65c222f6fa388b22d7cf80eb561a1f4) and [`compare`]() which show an image or display two of them side by side for comparison. We will be using both functions extensively throughout the tutorial, so you might want to check out their code I hyperlinked.

By convention, the third dimension of the ndarray is for the color channel but this convention isn't always followed. Whenever you find it so, Skimage usually provides parameters to specify this behavior. 

Images are unlike the usual Matplotlib plots. Their origin isn't located in the bottom left, but at the position `(0, 0)`, the top left.

In [ ]:
show(image, axis=True)

When we plot images in Matplotlib, axes denote the ordering of the pixels but we will usually be hiding them, since they don't deliver much value to the viewer.

<a href="#toc">Back to top🔝</a>

### 3. Common transformations <small id="common_trans"></small>

The most common image transformation we will be performing is converting color images to grayscale. Many image processing algorithms require grayscale, 2D arrays because color isn't the defining feature of images and computers can already extract enough information without it.

In [ ]:
from skimage.color import rgb2gray

image = imread("../input/notebook-images/grayscale_example.jpg")
# Convert image to grayscale
gray = rgb2gray(image)

compare(image, gray, "Grayscale Image")

In [ ]:
gray.shape

When we convert images to grayscale, they lose their 3rd dimension - the color channel. Instead, each cell in the image array now represents an integer in `uint8` type. They range from 0 to 255, giving 256 shades of gray. 

You can also use NumPy functions like [`np.flipud`](https://numpy.org/doc/stable/reference/generated/numpy.flipud.html) or [`np.fliplr`](https://numpy.org/doc/stable/reference/generated/numpy.fliplr.html#numpy.fliplr) at your heart's desire to manipulate images in any way a NumPy array can be manipulated.

In [ ]:
kitten = imread("../input/notebook-images/horizontal_flip.jpg")
horizontal_flipped = np.fliplr(kitten)

compare(kitten, horizontal_flipped, "Horizontally Flipped Image")

In [ ]:
ball = imread("../input/notebook-images/upside_down.jpg")
vertically_flipped = np.flipud(ball)

compare(ball, vertically_flipped, "Vertically Flipped Image")

In the [`color` module](https://scikit-image.org/docs/dev/api/skimage.color.html), you can find many other transformation functions to work with colors in images.

<a href="#toc">Back to top🔝</a>

### 4. Histogram of color channels <small id="histogram"></small>

Sometimes, it is useful to look at the intensity of each color channel to get a feel of the color distributions. We can do so by slicing each color channel and plotting their histograms. Here is a function to perform this operation:

In [ ]:
def plot_with_hist_channel(image, channel):
    channels = ["red", "green", "blue"]
    channel_idx = channels.index(channel)
    color = channels[channel_idx]

    extracted_channel = image[:, :, channel_idx]
    fig, (ax1, ax2) = plt.subplots(
        ncols=2, figsize=(18, 6)
    )  # , sharex=True, sharey=True)

    ax1.imshow(image)
    ax1.axis("off")
    ax2.hist(extracted_channel.ravel(), bins=256, color=color)
    ax2.set_title(f"{channels[channel_idx]} histogram")

Apart from the few Matplotlib details, you should pay attention to the call of the `hist` function. Once we extract the color channel and its array, we flatten it into 1D array and then pass it to the `hist` function. The number of bins should be 256, one for every pixel value - 0 being pitch black and 255 being fully white.

Let's use the function for our colorful scenery image:

In [ ]:
colorful_scenery = imread("../input/notebook-images/colorful_scenery.jpg")

plot_with_hist_channel(colorful_scenery, "red")

In [ ]:
plot_with_hist_channel(colorful_scenery, "green")

In [ ]:
plot_with_hist_channel(colorful_scenery, "blue")

You can also use histograms to find out the lightness in the image after converting it to a grayscale:

In [ ]:
gray_color_scenery = rgb2gray(colorful_scenery)

plt.hist(gray_color_scenery.ravel(), bins=256);

Most pixels have lower values as the scenery image is a bit darker.

We will explore more applications of histograms in the next sections.

<a href="#toc">Back to top🔝</a>

## Filters <small id="filters"></small>

### 1. Manual thresholding <small id="threshold_manual"></small>

Now, we arrive at the fun stuff - filtering images. The first operation we will learn is thresholding. Let's load an example image:

In [ ]:
stag = imread("../input/notebook-images/binary_example.jpg")

show(stag)

Thresholding has many applications in image segmentation, object detection, finding edges or contours, etc. It is mostly used to differentiate the background and foreground of an image.

Thresholding works best on high contrast grayscale images, so we will convert the stag image:

In [ ]:
# Convert to graysacle
stag_gray = rgb2gray(stag)

show(stag_gray)

We will start with basic manual thresholding and move on to automatic. 

First, we look at the mean value of all pixels in the gray image:

In [ ]:
stag_gray.mean()

> Note that the above gray image's pixels are normalized between 0 and 1 by dividing all their values by 256.  

We obtain a mean of 0.2 which gives us a preliminary idea for the threshold we might want to use. 

Now, we use this threshold to mask the image array. If the pixel value is lower than the threshold, its value becomes 0 - black or 1 - white if otherwise. In other words, we get a black and white, binary picture:

In [ ]:
# Set threshold
threshold = 0.35
# Binarize
binary_image = stag_gray > threshold

compare(stag, binary_image, "Binary image")

In this version, we can differentiate the outline of the stag more clearly. We can reverse the mask so that the background turns white:

In [ ]:
inverted_binary = stag_gray <= threshold

compare(stag, inverted_binary, "Binary image inverted")

<a href="#toc">Back to top🔝</a>

### 2. Thresholding - global <small id="global"></small>

While it might be fun try out different thresholds and seeing their effect on the image, we usually perform thresholding by using an algorithm, which we will be more robust than our eyeball estimates. 

There are many thresholding algorithms, so it might be hard to choose one. In this case, `skimage` has `try_all_threshold` function which runs 7 thresholding algorithms on the given *grayscale* image. Let's load an example and convert it:

In [ ]:
flower = imread("../input/notebook-images/global_threshold_ex.jpg")

flower_gray = rgb2gray(flower)

compare(flower, flower_gray)

We will see if we can refine the tulips' features by using thresholding:

In [ ]:
from skimage.filters import try_all_threshold

fig, ax = try_all_threshold(flower_gray, figsize=(10, 8), verbose=False)

As you can see, some algorithms work better while others are horrible on this image. The `otsu` algorithm looks better, so we will continue using it.

At this point, I want to draw your attention back to the original tulip image:

In [ ]:
show(flower)

The image has an uneven background because there is so much light coming from the window behind. We can confirm this by plotting a histogram of the gray tulip:

In [ ]:
plt.hist(flower_gray.ravel(), bins=256);

As expected, most pixels values are at the far end of the histogram, confirming that they are mostly bright. 

Why is this important? Depending on the lightness of an image, the performance of thresholding algorithms also changes. For this reason, thresholding algorithms are divided into two types:

1. Global - for images with even, uniform backgrounds
2. Local - for images with different levels of brightness in different regions of the image.

The tulip image goes into the second category because the right part of the image is much brighter than the other half, making its background uneven. We can't use a global thresholding algorithm on it and which was the reason why the performance of all algorithms in [`try_all_threshold`](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.try_all_threshold) was so poor. 

We will come back to the tulip example and local thresholding in just a bit. For now, we will load another examples with a much refined brightness and try to automatically threshold it:

In [ ]:
spiral = imread("../input/notebook-images/otsu_example.jpg")
spiral_gray = rgb2gray(spiral)

compare(spiral, spiral_gray)

We will use a common global tresholding algorithm [`threshold_otsu`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_otsu) in Skimage:

In [ ]:
from skimage.filters import threshold_otsu

# Find optimal threshold with `threshold_otsu`
threshold = threshold_otsu(spiral_gray)

# Binarize
binary_spiral = spiral_gray > threshold

compare(spiral, binary_spiral, "Binarized Image w. Otsu Thresholding")

Works much better!

### 3. Thresholding - local <small id="local"></small>

Now, we will work with local thresholding algorithms. 

Instead of looking at the whole image, local algorithms focus on pixel neighborhoods to account for the uneven brightness in different regions. A common local algorithm in `skimage` is given as [`threshold_local`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_local) function:

In [ ]:
from skimage.filters import threshold_local

local_thresh = threshold_local(flower_gray, block_size=3, offset=0.0002)

binary_flower = flower_gray > local_thresh

compare(flower, binary_flower, "Tresholded flower image")

You have to play around with the `offset` argument to find the optimal image to your needs. `offset` is the constant that is subtracted from the mean of the local pixel neighborhood. This "pixel neighborhood" is determined with the `block_size` parameter in `local_threshold`, which denotes the number of pixels the algorithm looks around each point in each direction.

Obviously, it is a disadvantage to tune both `offset` and `block_size` but local thresholding is the only option that yields better results than manual or global thresholding. 

Let's try one more example:

In [ ]:
from skimage.filters import threshold_local

handwriting = imread("../input/notebook-images/chalk_writing.jpg")
handwriting_gray = rgb2gray(handwriting)

# Find optimal threshold using local
local_thresh = threshold_local(handwriting_gray, offset=0.0003)

# Binarize
binary_handwriting = handwriting_gray > local_thresh

compare(handwriting, binary_handwriting, "Binarized image with local thresholding")

As you can see, the handwriting on the board is more refined after thresholding.

<a href="#toc">Back to top🔝</a>

### 4. Edge detection <small id="edge"></small>

Edge detection is useful in many ways, such as identifying objects, extracting features from them, counting them and many more. We will start with the basic Sobel filter, which finds edges of objects in gray scale images. We will load an image of coins and use the Sobel filter on them:

In [ ]:
from skimage.filters import sobel

coins = imread("../input/notebook-images/coins_2.jpg")
coins_gray = rgb2gray(coins)

coins_edge = sobel(coins_gray)

compare(coins, coins_edge, "Images of coins with edges detected")

The Sobel is pretty straightforward, you just have to call it on the gray image to get an output like above. We will see a more sophisticated version of Sobel in a later section. 

<a href="#toc">Back to top🔝</a>

### 5. Smoothing <small id="smooth"></small>

Another image filtering technique is smoothing. Many images like the chickens below, may contain random noise with no useful information to ML and DL algorithms. 

For example, the hairs around the chickens add noise to the image, which may deviate the attention of ML models from the main objects themselves. In such scenarios, we use smoothing to blur the noise or edges and reduce contrast. 

In [ ]:
chickens = imread("../input/notebook-images/chickens.jpg")

show(chickens)

One of the most popular and powerful smoothing techniques is [`gaussian`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian) smoothing:

In [ ]:
from skimage.filters import gaussian

smoothed = gaussian(chickens, multichannel=True, sigma=2)

compare(chickens, smoothed, "An image smoothed with Gaussian smoothing")

You can control the effect of the blur by tweaking the `sigma` argument. Don't forget to set `multichannel` to True if you are dealing with an RGB image. 

If the image resolution is too high, the smoothing effect might not be visible to the naked eye but it will still be pronounced under the hood.

<a href="#toc">Back to top🔝</a>

### 6. Contrast enhancement <small id="contrast"></small>

Certain types of images like medical analysis results have low contrast, making it hard to spot details, like below:

In [ ]:
xray = imread("../input/notebook-images/xray.jpg")
xray_gray = rgb2gray(xray)

compare(xray, xray_gray)

In such scenarios, we can use contrast enhancement to make the details more distinct. There are two types of contrast enhancement algorithms:

1. Contrast stretching
2. Histogram equalization

We will discuss histogram equalization in this post, which, in turn, has three types:

1. Standard histogram equalization
2. Adaptive histogram equalization
3. Contrast Limited Adaptive Histogram Equalization (CLAHE)

[Histogram equalization](https://en.wikipedia.org/wiki/Histogram_equalization) spreads out the areas with the highest contrast of an image to less bright regions, *equalizing it*.

> Oh, by the way, you can calculate the contrast of an image by subtracting the lowest pixel value from the highest.

In [ ]:
xray.max() - xray.min()

Now, let's try the standard histogram equalization from the `exposure` module:

In [ ]:
from skimage.exposure import equalize_hist

enhanced = equalize_hist(xray_gray)

compare(xray, enhanced)

We can already see the details a lot more clearly. 

Next, we will use the CLAHE (this is a fun word to pronounce!) which computes many histograms for different pixel neighborhoods in an image, which results more detail even in the darkest of the regions:

In [ ]:
from skimage.exposure import equalize_adapthist

# Adjust clip_limit
enhanced_adaptive = equalize_adapthist(xray_gray, clip_limit=0.4)

compare(xray, enhanced_adaptive, "Image with contrast enhancement")

This one looks a lot better since it could show details in the background and also a couple more missing ribs in the bottom left. You can tweak `clip_limit` for more or less detail. 

<a href="#toc">Back to top🔝</a>

### 7. Transformations <small id="transformations"></small>

Images in your dataset might have several clashing characteristics, like different scales, unaligned rotations, etc. ML and DL algorithms expect your images to be of the same shape and dimensions. Therefore, you need to learn to how fix them.

**Rotations**

To rotate images, use the `rotate` function from the `transform` module. I've chosen actual clocks so you might remember the angle signs better:

In [ ]:
from skimage.transform import rotate

clock = imread("../input/notebook-images/clock.jpg")

clockwise = rotate(clock, angle=-60)
compare(clock, clockwise, "Clockwise rotated image, use negative angles")

In [ ]:
anti_clockwise = rotate(clock, angle=33)

compare(clock, anti_clockwise, "Anticlockwise rotated image, use positive angles")

**Rescaling**

Another common operation is scaling images. It is mostly useful in cases where images are proportionally different from one another. 

We use the similar [`rescale`](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.rescale) function for this operation:

In [ ]:
butterflies = imread("../input/notebook-images/butterflies.jpg")
butterflies.shape

In [ ]:
from skimage.transform import rescale

scaled_butterflies = rescale(butterflies, scale=3 / 4, multichannel=True)

compare(
    butterflies,
    scaled_butterflies,
    "Butterflies scaled down by a factor of 3/4",
    axis=True,
)

When image resolution is high, downscaling it too much might result in quality loss or pixels rubbing together unceremoniously to create unexpected edges or corners. To account for this effect, you can set `anti_aliasing` to True which uses Gaussian smoothing under the hood:

In [ ]:
factor_10_aa = rescale(butterflies, scale=1 / 10, multichannel=True, anti_aliasing=True)
factor_10_no_aa = rescale(butterflies, scale=1 / 10, multichannel=True)

compare(factor_10_aa, factor_10_no_aa, "No anti-aliasing")

As before, the smoothing isn't noticeable but at a more granular level, it will be obvious.

**Resizing**

If you want the image to have specific width and height, rather than scaling it by a factor, you can use the [`resize`](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.resize) function by providing an `output_shape`:

In [ ]:
from skimage.transform import resize

puppies = imread("../input/notebook-images/puppies.jpg")

# Also possible to set anti_aliasing
puppies_600_800 = resize(puppies, output_shape=(600, 800))

compare(puppies, puppies_600_800, "Puppies image resized 600x800 (height, width)")

<a href="#toc">Back to top🔝</a>

## Image restoration and enhancement <small id="restoration"></small>

Some images might be distorted, damaged or lost during file transforms, in faulty downloads or in many other situations. Rather than giving up on the image, you can use `skimage` to account for the damage and make the image good as new. 

In this section, we will discuss a few techniques for image restoration, starting with inpainting.

### 1. Inpainting <small id="inpainting"></small>

An inpainting algorithm can intelligently fill in the blanks in an image. I couldn't find a damaged image, so we will use this whale image and put a few blanks on it manually:

In [ ]:
whale_image = imread("../input/happy-whale-and-dolphin/train_images/00206a224e68de.jpg")

show(whale_image)

In [ ]:
whale_image.shape

The below function creates four pitch black regions to simulate lost information on an image:

In [ ]:
def make_mask(image):
    """Create a mask to artificially defect the image."""

    mask = np.zeros(image.shape[:-1])

    # Make 4 masks
    mask[250:300, 1400:1600] = 1
    mask[50:100, 300:433] = 1
    mask[300:380, 1000:1200] = 1
    mask[200:270, 750:950] = 1

    return mask.astype(bool)

In [ ]:
# Create the mask
mask = make_mask(whale_image)

# Apply the defect mask on the whale_image
image_defect = whale_image * ~mask[..., np.newaxis]

In [ ]:
compare(whale_image, image_defect, "Artifically damaged image of a whale")

We will use the [`inpaint_biharmonic`](https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.inpaint_biharmonic) function from the `inpaint` module to fill in the blanks, passing in the `mask` we created:

In [ ]:
from skimage.restoration import inpaint

restored_image = inpaint.inpaint_biharmonic(
    image=image_defect, mask=mask, multichannel=True
)

compare(
    image_defect,
    restored_image,
    "Restored image after defects",
    title_original="Faulty Image",
)

As you can see, it will be hard to tell where the defect regions are before seeing the faulty image.

Now, let's make some noise📣!

<a href="#toc">Back to top🔝</a>

### 2. Noise📣 <small id="noise"></small>

As discussed earlier, noise plays important role in image enhancement and restoration. Sometimes, you might intentionally add it to an image like below:

In [ ]:
from skimage.util import random_noise

pup = imread("../input/notebook-images/pup.jpg")

noisy_pup = random_noise(pup)

compare(pup, noisy_pup, "Noise puppy image")

We use the [`random_noise`](https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.random_noise) function to sprinkle an image with random specks of color. For this reason, the method is called "salt and pepper" technique.

<a href="#toc">Back to top🔝</a>

### 3. Reducing noise - denoising <small id="denoise"></small>

But, most of the time, you want to remove noise from an image, rather than add it. There are a few types of denoising algorithms:

1. Total variation (TV) filter
2. Bilateral denoising
3. Wavelet denoising 
4. Non-local mean denoising

We will only look at the first two in this article. Let's try TV filter first, which is available as [`denoise_tv_chambolle`](https://scikit-image.org/docs/stable/api/skimage.restoration.html#denoise-tv-chambolle):

In [ ]:
from skimage.restoration import denoise_tv_chambolle

denoised_pup_tv = denoise_tv_chambolle(noisy_pup, weight=0.2, multichannel=True)

compare(
    noisy_pup,
    denoised_pup_tv,
    "Total Variation Filter denoising applied",
    title_original="Noisy pup",
)

The higher the resolution of the image, the longer it takes to denoise it. You can control the effect of denoising with the `weight` parameter. Now, let's try [`denoise_bilateral`](https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_bilateral):

In [ ]:
from skimage.restoration import denoise_bilateral

denoised_pup_bilateral = denoise_bilateral(noisy_pup, multichannel=True)

compare(noisy_pup, denoised_pup_bilateral, "Bilateral denoising applied image")

It wasn't as effective as TV filter as can be seen below:

In [ ]:
compare(
    denoised_pup_tv,
    denoised_pup_bilateral,
    "Bilateral filtering",
    title_original="TV filtering",
)

<a href="#toc">Back to top🔝</a>

### 4. Superpixels and Segmentation <small id="superpixel"></small>

Image segmentation is one of the most fundamental and common topics in image processing. It is extensively used in motion and object detection, image classification and many more areas. 

We've already seen an instance of segmentation - thesholding an image so that the background is extracted from the foreground. In this section, we will learn to do more than that such as segmenting image into similar areas.

To get started with segmentation, we need to understand a concept of superpixels.

A pixel, on its own, just represents a small area of color. Once separated from the image, a single pixel will be useless. For this reason, segmentation algorithms use multiple groups of pixels that are similar in contrast, color or brightness. They are called superpixels. 

One of the algorithms that tries to find superpixels is Simple Linear Iterative Cluster (SLIC), which uses k-Means clustering under the hood. Let's see how to use it on the coffee image available in the `skimage` library:

In [ ]:
from skimage import data

coffee = data.coffee()

show(coffee)

We will use the [`slic`](https://scikit-image.org/docs/dev/api/skimage.segmentation.html?highlight=slic#skimage.segmentation.slic) function from the `segmentation` module:

In [ ]:
from skimage.segmentation import slic

segments = slic(coffee)
show(segments)

`slic` finds 100 segments or labels by default. To put them back onto the image, we use the [`label2rgb`](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.label2rgb) function:

In [ ]:
from skimage.color import label2rgb

final_image = label2rgb(segments, coffee, kind="avg")
show(final_image)

Let's wrap this operation inside a function and try to use more segments:

In [ ]:
from skimage.color import label2rgb
from skimage.segmentation import slic


def segment(image, n_segments=100):
    # Obtain superpixels / segments
    superpixels = slic(coffee, n_segments=n_segments)

    # Put the groups on top of the original image
    segmented_image = label2rgb(superpixels, image, kind="avg")

    return segmented_image

In [ ]:
# Find 500 segments
coffee_segmented_2 = segment(coffee, n_segments=500)

compare(coffee, coffee_segmented_2, "With 500 segments")

Segmentation will make it easier for computer vision algorithms to extract useful features from images.

<a href="#toc">Back to top🔝</a>

### 5. Contours <small id="contour"></small>

Much of the information of an object resides in its shape. If we can detect an object's shape in lines or in contours, we can extract many useful data like object's size, its individual markings, etc.

Let's see finding contours in practice using the image of dominoes. 

In [ ]:
dominoes = imread("../input/notebook-images/dominoes.jpg")

show(dominoes)

We will see if we can isolate the tiles and circles using the [`find_contours`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.find_contours) function in `skimage`. This function requires a binary (black and white) image, so we have to threshold the image first. 

In [ ]:
from skimage.measure import find_contours

# Convert to grayscale
dominoes_gray = rgb2gray(dominoes)
# Find optimal threshold with treshold_otsu
thresh = threshold_otsu(dominoes_gray)
# Binarize
dominoes_binary = dominoes_gray > thresh

domino_contours = find_contours(dominoes_binary)

The resulting array is a list of (n, 2) arrays representing the coordinates of the contour lines:

In [ ]:
for contour in domino_contours[:5]:
    print(contour.shape)

We will wrap the operation inside a function called `mark_contours`:

In [ ]:
from skimage.filters import threshold_otsu
from skimage.measure import find_contours


def mark_contours(image):
    """A function to find contours from an image"""
    gray_image = rgb2gray(image)
    # Find optimal threshold
    thresh = threshold_otsu(gray_image)
    # Mask
    binary_image = gray_image > thresh

    contours = find_contours(binary_image)

    return contours

To plot the contour lines on the image, we will create another function called `plot_image_contours` that uses the above one:

In [ ]:
def plot_image_contours(image):
    fig, ax = plt.subplots()

    ax.imshow(image, cmap=plt.cm.gray)

    for contour in mark_contours(image):
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color="red")

    ax.axis("off")


plot_image_contours(dominoes)

As we can see, we successfully detected the majority of the contours, but we can still see some random fluctuations in the center. Let's apply denoising before we pass the image of dominoes to our contour finding function:

In [ ]:
dominoes_denoised = denoise_tv_chambolle(dominoes, multichannel=True)

plot_image_contours(dominoes_denoised)

That's it! We eliminated most of the noise which was causing the incorrect contour lines!

<a href="#toc">Back to top🔝</a>

## Advanced operations <small id="advanced"></small>

### 1. Edge detection <small id="edge2"></small>

Before, we used the Sobel algorithm to detect edges of objects. Here, we will use the Canny algorithm which is more widely used for it is more faster and more accurate. As always, the function [`canny`](https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.canny) requires a grayscale image. 

This time we will use an image with more coins, hence more edges to detect:

In [ ]:
coins_3 = imread("../input/notebook-images/coins_3.jpg")

# Convert to gray
coins_3_gray = rgb2gray(coins_3)

compare(coins_3, coins_3_gray)

To find edges, we just pass the image to the `canny` function:

In [ ]:
from skimage.feature import canny

# Find edges with canny
canny_edges = canny(coins_3_gray)

compare(coins_3, canny_edges, "Edges detected with Canny algorithm")

The algorithm certainly found almost all coins edges but it is very noisy because the engravings on the coins are detected as well. We can reduce the sensitivity of `canny` by tweaking the `sigma` parameter:

In [ ]:
canny_edges_sigma_2 = canny(coins_3_gray, sigma=2.5)

compare(coins_3, canny_edges_sigma_2, "Edges detected using Canny with less intensity")

As you can see, `canny` now only finds the general outline of the coins. 

<a href="#toc">Back to top🔝</a>

### 2. Corner detection <small id="corner"></small>

Another important image processing technique is corner detection. Corners can be key features of objects in image classification. 

To find corners, we will use the Harris corner detection algorithm. Let's load a sample image and convert it to grayscale:

In [ ]:
windows = imread("../input/notebook-images/windows.jpg")

windows_gray = rgb2gray(windows)

compare(windows, windows_gray)

We will use the [`corner_harris`](https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.corner_harris) function to produce a measure image that masks the areas where corners are. 

In [ ]:
from skimage.feature import corner_harris

measured_image = corner_harris(windows_gray)

show(measured_image)

Now, we will pass this masked measure image to `corner_peaks` function, which returns corner coordinates this time:

In [ ]:
from skimage.feature import corner_peaks

corner_coords = corner_peaks(measured_image, min_distance=50)
len(corner_coords)

The function found 79 corners using a minimum distance of 50 pixels between each corner. Let's wrap the operation up to this point in a function:

In [ ]:
def find_corner_coords(image, min_distance=50):
    # Convert to gray
    gray_image = rgb2gray(image)
    # Produce a measure image
    measure_image = corner_harris(gray_image)

    # Find coords
    coords = corner_peaks(measure_image, min_distance=min_distance)

    return coords

Now, we will create another function that plots each corner using the coordinates produced from the above function:

In [ ]:
def show_image_cornered(image):
    # Find coords
    coords = find_corner_coords(image)

    # Plot them on top of the image
    plt.imshow(image, cmap="gray")
    plt.plot(coords[:, 1], coords[:, 0], "+b", markersize=15)
    plt.axis("off")


show_image_cornered(windows)

Unfortunately, the algorithm isn't working as expected. Rather than finding the window corners, the marks are placed at the intersection of the bricks. These intersections are noise, making them useless. Let's denoise the image and pass it to the function once again:

In [ ]:
windows_denoised = denoise_tv_chambolle(windows, multichannel=True, weight=0.3)

show_image_cornered(windows_denoised)

Now, this is much better! It ignored the brick edges and found the majority of window corners.

<a href="#toc">Back to top🔝</a>

### Conclusion

Phew! What a post! Both you and me deserve a pat on the back!

I had quite fun writing this post. In a real computer vision problem, you won't be using all of these at once, of course. As you may have noticed, things we learned today aren't very difficult. They take a few lines of code, at most. The difficult part is applying them to a real problem and actually improving the performance of your model.

That bit comes with hard work and practice, not nicely packaged inside a single article. Thank you for reading!

**You can become a premium Medium member using the link below and get access to all of my stories and thousands of others:**

https://ibexorigin.medium.com/membership

**Or subscribe to my email list:**

https://ibexorigin.medium.com/subscribe

**You can reach out to me on [LinkedIn](https://twitter.com/BexTuychiev) or [Twitter](https://twitter.com/BexTuychiev) for a friendly chat about all things data.**

![](https://cdn-images-1.medium.com/max/900/1*KeMS7gxVGsgx8KC36rSTcg.gif)